In [99]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [100]:
# Load the CSV data
data = pd.read_csv("Texas_Month_Wise_Data/Texas_Yield_Combined_Data.csv")
#drop coordinated column from dataframe
data = data.drop(['coordinates'], axis=1)
# Convert "Date" column to proper date format
data['date'] = pd.to_datetime(data['date'])

In [69]:
data

,field_id,year,Area,date,day time temperature,night time temperature,Field Sentinel 2 NDVI,Field Sentinel 2 EVI,Field Sentinel 2 GNDVI,Field Sentinel 2 SAVI,...,Field NDBaI_8,Field MNDWI_8,Field NDVI_8,Field GNDVI_8,Field EVI_8,Field MODIS LAI,Field MODIS NDVI,Field MODIS EVI,Field MOISTURE,Annual Yield
0,1.0,2019.0,3.564942,2019-01-02,302.44,285.12,-0.020757,0.056363,-0.054122,-0.031134,...,0.012377,0.373777,-0.010655,-0.017766,0.045364,3.000000,-0.326334,-0.421288,4.448739,136.199314
1,1.0,2019.0,3.564942,2019-01-03,305.60,292.94,0.094908,-0.169957,0.065104,0.142350,...,0.144259,0.158599,-0.000238,0.004833,0.001578,8.000000,0.121314,-0.191442,12.510705,136.199314
2,1.0,2019.0,3.564942,2019-01-04,310.44,292.90,0.337292,0.731048,0.290686,0.505885,...,0.067448,-0.213212,0.233985,0.228786,1.974063,9.000000,0.276460,-0.105611,1.995070,136.199314
3,1.0,2019.0,3.564942,2019-01-05,311.32,300.16,0.374844,0.913023,0.289348,0.562200,...,0.124935,-0.173063,0.396155,0.359396,2.513485,6.000000,0.580167,0.255000,2.962755,136.199314
4,1.0,2019.0,3.564942,2019-01-06,316.56,300.70,0.380820,1.537087,0.318652,0.571170,...,0.151209,-0.151297,0.415790,0.376995,2.023017,17.000000,0.555500,0.393167,7.173752,136.199314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13495,450.0,2022.0,0.222810,2022-07-01,316.08,302.04,0.377543,2.084189,0.346872,0.566273,...,0.087391,-0.188673,0.230694,0.232797,0.982791,5.488449,0.016500,-0.221863,2.313678,5.523465
13496,450.0,2022.0,0.222810,2022-08-01,315.98,302.02,0.521138,2.983756,0.448448,0.781652,...,-0.068380,0.413185,-0.025999,-0.028093,0.154592,15.534653,0.308455,0.101070,2.226456,5.523465
13497,450.0,2022.0,0.222810,2022-09-01,310.88,298.38,0.305374,2.694511,0.263938,0.458011,...,0.058576,-0.072215,0.209206,0.182903,4.429068,6.023102,0.066649,-0.065198,2.226456,5.523465
13498,450.0,2022.0,0.222810,2022-10-01,309.30,291.64,0.154149,1.181334,0.175074,0.231210,...,0.130448,-0.173786,0.338615,0.318038,2.166802,7.023102,0.251931,-0.088648,2.226456,5.523465


In [101]:
data.index = data.pop('date')

In [71]:
data

,field_id,year,Area,day time temperature,night time temperature,Field Sentinel 2 NDVI,Field Sentinel 2 EVI,Field Sentinel 2 GNDVI,Field Sentinel 2 SAVI,Field Sentinel 2 B1,...,Field NDBaI_8,Field MNDWI_8,Field NDVI_8,Field GNDVI_8,Field EVI_8,Field MODIS LAI,Field MODIS NDVI,Field MODIS EVI,Field MOISTURE,Annual Yield
date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,1.0,2019.0,3.564942,302.44,285.12,-0.020757,0.056363,-0.054122,-0.031134,8897.028578,...,0.012377,0.373777,-0.010655,-0.017766,0.045364,3.000000,-0.326334,-0.421288,4.448739,136.199314
2019-01-03,1.0,2019.0,3.564942,305.60,292.94,0.094908,-0.169957,0.065104,0.142350,3206.411519,...,0.144259,0.158599,-0.000238,0.004833,0.001578,8.000000,0.121314,-0.191442,12.510705,136.199314
2019-01-04,1.0,2019.0,3.564942,310.44,292.90,0.337292,0.731048,0.290686,0.505885,1719.698505,...,0.067448,-0.213212,0.233985,0.228786,1.974063,9.000000,0.276460,-0.105611,1.995070,136.199314
2019-01-05,1.0,2019.0,3.564942,311.32,300.16,0.374844,0.913023,0.289348,0.562200,1593.131457,...,0.124935,-0.173063,0.396155,0.359396,2.513485,6.000000,0.580167,0.255000,2.962755,136.199314
2019-01-06,1.0,2019.0,3.564942,316.56,300.70,0.380820,1.537087,0.318652,0.571170,2313.553638,...,0.151209,-0.151297,0.415790,0.376995,2.023017,17.000000,0.555500,0.393167,7.173752,136.199314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-01,450.0,2022.0,0.222810,316.08,302.04,0.377543,2.084189,0.346872,0.566273,2044.574257,...,0.087391,-0.188673,0.230694,0.232797,0.982791,5.488449,0.016500,-0.221863,2.313678,5.523465
2022-08-01,450.0,2022.0,0.222810,315.98,302.02,0.521138,2.983756,0.448448,0.781652,1714.811881,...,-0.068380,0.413185,-0.025999,-0.028093,0.154592,15.534653,0.308455,0.101070,2.226456,5.523465
2022-09-01,450.0,2022.0,0.222810,310.88,298.38,0.305374,2.694511,0.263938,0.458011,1721.912541,...,0.058576,-0.072215,0.209206,0.182903,4.429068,6.023102,0.066649,-0.065198,2.226456,5.523465


In [72]:
#dataframe column types
data.dtypes

field_id                      float64
year                          float64
Area                          float64
day time temperature          float64
night time temperature        float64
Field Sentinel 2 NDVI         float64
Field Sentinel 2 EVI          float64
Field Sentinel 2 GNDVI        float64
Field Sentinel 2 SAVI         float64
Field Sentinel 2 B1           float64
Field Sentinel 2 B2           float64
Field Sentinel 2 B3           float64
Field Sentinel 2 B4           float64
Field Sentinel 2 B5           float64
Field Sentinel 2 B6           float64
Field Sentinel 2 B7           float64
Field Sentinel 2 B8           float64
Field Sentinel 2 B9           float64
Field Sentinel 2 B11          float64
Field Sentinel 2 B12          float64
Field Sentinel 2 VCI          float64
Field Sentinel 2 TVI          float64
Field Sentinel 2 BI           float64
Field Sentinel 2 BI2          float64
Field Sentinel 2 CI           float64
Field Sentinel 2 CI1          float64
Field Sentin

In [102]:
# Step 2: Prepare the input sequences
def create_sequences(data, window_size):
    sequences = []
    for point_id, group in data.groupby('field_id'):
        values = group.drop(['Annual Yield', 'field_id'], axis=1).values
        num_sequences = len(values) - window_size + 1
        for i in range(num_sequences):
            sequence = values[i:i+window_size]
            sequences.append((point_id, sequence))
    return sequences

In [103]:
window_size = 11  # You can adjust the window size based on your data and preference
sequences = create_sequences(data, window_size)

In [105]:
# Split the data into training and testing sets
train_sequences, test_sequences = train_test_split(sequences, test_size=0.2, random_state=42)

In [123]:
# Step 3: Build and train the LSTM model
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64 ,input_shape=input_shape))
    model.add(Dense(1))  # Single output for regression

    # Compile the model
    model.compile(loss="mean_squared_error", optimizer="adam")
    return model

In [117]:
# Prepare the training data
X_train = np.array([seq for _, seq in train_sequences])
y_train = np.array([data[data['field_id'] == point_id]['Annual Yield'].iloc[-1] for point_id, _ in train_sequences])

# Prepare the testing data
X_test = np.array([seq for _, seq in test_sequences])
y_test = np.array([data[data['field_id'] == point_id]['Annual Yield'].iloc[-1] for point_id, _ in test_sequences])

In [118]:
# Standardize the input data for better training
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

In [124]:
# Build and train the LSTM model
input_shape = (window_size, X_train.shape[2])
model = build_lstm_model(input_shape)
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/5
203/203 [==============================] - 5s 11ms/step - loss: nan - val_loss: nan
Epoch 2/5
203/203 [==============================] - 2s 10ms/step - loss: nan - val_loss: nan
Epoch 3/5
203/203 [==============================] - 2s 9ms/step - loss: nan - val_loss: nan
Epoch 4/5
203/203 [==============================] - 2s 10ms/step - loss: nan - val_loss: nan
Epoch 5/5
203/203 [==============================] - 2s 8ms/step - loss: nan - val_loss: nan


In [58]:
model.evaluate(X_test,y_test)

57/57 [==============================] - 0s 7ms/step - loss: nan


nan